In [ ]:
!pip install -U spacy
!pip install spacy_transformers
!pip install pymupdf

In [ ]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import sys,fitz
import os

In [ ]:
nlp = spacy.load('./output/model-best')

In [ ]:
fname = input("Enter path of folder:")
data_of_applicants=[]
def get_files_in_folder(fname):
    files = []
    for file_name in os.listdir(fname):
        file_path = os.path.join(fname, file_name)
        if os.path.isfile(file_path):
            files.append(file_path)
    return files
idnum=0
for applicant in get_files_in_folder(fname):
    doc1 = fitz.open(applicant)
    text = " "
    idnum+=1
    applicant_data={}
    for page in doc1:
        text = text + str(page.get_text())
    doc1.close()
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ in applicant_data:
            applicant_data[ent.label_].append(ent.text)
        else:
            applicant_data["ID"]=idnum
            applicant_data[ent.label_]=[ent.text]
    data_of_applicants.append(applicant_data)


In [ ]:
data_of_applicants

In [ ]:
job_description={}
def input_job_description():
    skills = input("Enter skills (comma-separated): ").split(',')
    experience = input("Enter experience (comma-separated): ").split(',')
    languages = input("Enter languages (comma-separated): ").split(',')
    location = input("Enter location: ").split()
    worked_as = input("Enter worked as (comma-separated): ").split(',')
    degree = input("Enter degree (comma-separated): ").split(',')
    job_description['SKILLS']=skills
    job_description['WORKED AS']=worked_as
    job_description['LANGUAGE']=languages
    job_description['LOCATION']=location
    job_description['YEARS OF EXPERIENCE']=experience
    job_description['DEGREE']=degree
    return job_description

In [ ]:
job_description=input_job_description()

In [ ]:
job_description

In [ ]:
def calculate_similarity_and_score(data1, data2):
    nlp_model = spacy.load("en_core_web_md")
    
    skills_data1 = ' '.join(data1.get('SKILLS', []))
    skills_data2 = ' '.join(data2.get('SKILLS', []))
    
    degree_data1 = ' '.join(data1.get('DEGREE', []))
    degree_data2 = ' '.join(data2.get('DEGREE', []))
    
    location_data1 = ' '.join(data1.get('LOCATION', []))
    location_data2 = ' '.join(data2.get('LOCATION', []))
    
    experience_data1 = ' '.join(data1.get('YEARS OF EXPERIENCE', []))
    experience_data2 = ' '.join(data2.get('YEARS OF EXPERIENCE', []))
    
    language_data1 = ' '.join(data1.get('LANGUAGE', []))
    language_data2 = ' '.join(data2.get('LANGUAGE', []))
    
    worked_as_data1 = ' '.join(data1.get('WORKED AS', []))
    worked_as_data2 = ' '.join(data2.get('WORKED AS', []))
    
    skills_similarity = nlp_model(skills_data1).similarity(nlp_model(skills_data2))
    degree_similarity = nlp_model(degree_data1).similarity(nlp_model(degree_data2))
    location_similarity = nlp_model(location_data1).similarity(nlp_model(location_data2))
    experience_similarity = nlp_model(experience_data1).similarity(nlp_model(experience_data2))
    language_similarity = nlp_model(language_data1).similarity(nlp_model(language_data2))
    worked_as_similarity = nlp_model(worked_as_data1).similarity(nlp_model(worked_as_data2))
    
    similarity_scores = [skills_similarity, degree_similarity, location_similarity, experience_similarity, language_similarity, worked_as_similarity]
    overall_score = sum(similarity_scores) / len(similarity_scores)
    
    return {'ID': data1['ID'], 'diff_scores': similarity_scores, 'overall': overall_score}

In [ ]:
all_scores=[]
for applicant_data_comp in data_of_applicants:
    all_scores.append(calculate_similarity_and_score(applicant_data_comp,job_description))

sorted_scores = sorted(all_scores, key=lambda x: x['overall'], reverse=True)

In [ ]:
sorted_scores

In [ ]:
def print_top_applicants(data_of_applicants, sorted_scores, top_n=3):
    for i in range(min(top_n, len(sorted_scores))):
        applicant_id = sorted_scores[i]['ID']
        for applicant in data_of_applicants:
            if applicant['ID'] == applicant_id:
                print(f"Details of applicant with ID {applicant_id}:")
                for key, value in applicant.items():
                    print(f"{key}: {value}")
                print()

print_top_applicants(data_of_applicants, sorted_scores)
    
